In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

transactions = pd.read_csv('Home_Challenge_Data_Source.csv')
transactions.created_at = pd.to_datetime(transactions.created_at, dayfirst=True)

transactions.to_sql('transactions', conn, if_exists='replace')

18408

## First transactions from new merchant_ids that happent at the last day ofmonth

In [4]:
query = """
SELECT 
    substring(first_date, 0, 11) as date_of_first_transaction,
    min(first_date) as earliest_transaction
FROM (
    SELECT 
        merchant_id, 
        min(created_at) as first_date
    FROM transactions
    group by merchant_id
)
where first_date like '%-31%'
group by substring(first_date, 0, 11)
"""
pd.read_sql_query(query, conn)

,date_of_first_transaction,earliest_transaction
0,2022-07-31,2022-07-31 02:01:00
1,2022-08-31,2022-08-31 04:26:00


## SQL queries

In [5]:
query = """
SELECT COUNT(DISTINCT merchant_id)
FROM transactions
WHERE 
    fee_type = 'reduced_fee_promo'
    and transaction_status = 'successful'
    and created_at like '2022-08%'
"""
text = 'merchants made transactions with the reduced fees in August 2022'
print(c.execute(query).fetchall()[0][0], text)

88 merchants made transactions with the reduced fees in August 2022


In [6]:
query = """
SELECT merchant_id
FROM transactions
WHERE 
    fee_type = 'reduced_fee_promo'
    and transaction_status = 'successful'
GROUP BY merchant_id
ORDER BY count() DESC
LIMIT 1
"""
print('Merchant_id', c.execute(query).fetchall()[0][0], 
      'had the highest number of successful reduced fee transactions')

Merchant_id 255 had the highest number of successful reduced fee transactions


## Bonus. Anomalies in data
1. Transaction below has reduced _fee_amount_eur_, but _fee_type_ indicated as default

In [7]:
query = """
SELECT *
FROM transactions
where fee_amount_eur != 1 and fee_type = 'default'

"""
pd.read_sql_query(query, conn)

,index,merchant_id,product,fee_type,fee_amount_eur,created_at,transaction_amount_eur,transaction_status
0,11918,280,card_reader,default,0.8,2022-08-18 04:36:00,67.3,successful


In [8]:
query = """
SELECT used_promo, not_used_promo, count() as merchants
FROM (
    SELECT merchant_id, max(fee_type = 'reduced_fee_promo') as used_promo, max(fee_type = 'default') as not_used_promo
    FROM transactions
    group by merchant_id
)
group by used_promo, not_used_promo
"""
pd.read_sql_query(query, conn)

,used_promo,not_used_promo,merchants
0,0,1,144
1,1,0,119
2,1,1,1


2. No transactions at 30 June

In [9]:
query = """
SELECT *
FROM transactions
where created_at like '2022-06-30%'

"""
pd.read_sql_query(query, conn)

,index,merchant_id,product,fee_type,fee_amount_eur,created_at,transaction_amount_eur,transaction_status


3. No new merchant_id for almost 2 months: since 6 June till 30 July

In [10]:
query = """
SELECT first_date, count() as new_merchants
FROM (
    SELECT merchant_id, substring(min(created_at), 0, 11) as first_date
    FROM transactions
    group by merchant_id
)
group by first_date
order by first_date
"""
pd.read_sql_query(query, conn)

,first_date,new_merchants
0,2022-06-01,69
1,2022-06-02,25
2,2022-06-03,4
3,2022-06-04,2
4,2022-06-05,1
5,2022-07-31,39
6,2022-08-01,28
7,2022-08-02,18
8,2022-08-03,19
9,2022-08-04,9
